In [7]:
import pandas as pd
from chat_bots import ChatSentimento, ChatFundamentalistas, ChatAnaliseTecnica, ChatBot
from coleta_dados import DadosFundamentalistas, DadosNoticiasBuscadorYahoo, DadosNoticiasGoogle
from tratando_dados import tratando_dados_fundamentalistas, TratarDadosNoticias, TratandoDadosIndicadores
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
import ipeadatapy as ip
from datetime import datetime
import pandas as pd
import yfinance as yf
import sidrapy
load_dotenv()

True

# Fundamentalista

In [129]:
tic = 'BBDC3'
data_inicio = '2020-01-01'

dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)
trat = tratando_dados_fundamentalistas(tic, data_inicio)

In [5]:
trat

[Document(metadata={'receita_liquida': 52500.0, 'resultado_bruto': 15210.0, 'lucro_liquido': 4870.0, 'tic': 'BBDC3', 'capex_tres_meses': -10.25, 'fluxo_caixa_operacional': -6280.0, 'fluxo_caixa_investimentos': 8500.0, 'fluxo_caixa_financiamento': -4050.0, 'aumento_reducao_caixa_equivalentes': -1930.0, 'preço_lucro': '9.32', 'preço_receita_líquida': '0.57', 'preço_fco': '-3.30', 'preço_ativo_total': '0.07', 'market_cap_empresa': 144240.0, 'dividend_yield': 0.1076999999999999, 'ativo_total': 2020000.0, 'patrimonio_liquido': 167130.0, 'valor_patrimonial_acao': '15.70', 'retorno_sobre_patrimonio_liquido_inicial': 0.088, 'retorno_sobre_ativo_inicial': 0.008, 'margem_bruta': 0.277, 'margem_liquida': 0.061, 'giro_do_ativo_inicial': '0.13', 'alavancagem_financeira': '12.07', 'passivo_patrimonio_liquido': '11.07'}, page_content='2024-09-30'),
 Document(metadata={'receita_liquida': 51500.0, 'resultado_bruto': 16940.0, 'lucro_liquido': 4120.0, 'tic': 'BBDC3', 'capex_tres_meses': -1190.0, 'fluxo_c

In [6]:
query = 'Como esta a saude financeira da bbdc3'
response = ChatFundamentalistas(query=query, dados=trat,modelo_llm="deepseek-r1-distill-llama-70b")

In [7]:
response.split('\n\n')

['<think>\nOk, preciso analisar a saúde financeira da BBDC3 com base nos indicadores fornecidos. Vou começar identificando as informações disponíveis e entendendo o que cada uma delas significa.',
 'Primeiro, vou olhar para a receita líquida. Vejo que ela tem flutuado um pouco, mas parece estar em uma tendência de crescimento gradual. Isso é positivo porque indica que a empresa está aumentando sua receita ao longo do tempo.',
 'Em seguida, o lucro líquido. Notando que o lucro líquido também tem variações, mas em alguns períodos foi bastante alto, como 7.870,00, mas em outros foi muito baixo, como 1.440,00. Isso pode indicar volatilidade nos resultados, o que pode ser preocupante.',
 'Margens brutas e líquidas estão estáveis, o que é bom. A margem bruta em torno de 27% e a líquida em torno de 5-6% indicam que a empresa mantém uma rentabilidade consistente, embora possa haver espaço para melhoria na margem líquida.',
 'Agora, olhando para os fluxos de caixa. O fluxo operacional tem sido 

# Sentimento

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

dados_yahoo = DadosNoticiasBuscadorYahoo(acao='BBDC3', options=chrome_options)


In [3]:
dados_ya = dados_yahoo.get_news(number_paginas=1)

Sem mais páginas!


In [12]:
dados_ya

{'links': ['https://www.infomoney.com.br/onde-investir/agenda-de-dividendos-de-fevereiro-2025/',
  'https://einvestidor.estadao.com.br/mercado/ibovespa-hoje-magazine-luiza-mglu3-salta-so-4-acoes-fecham-queda/',
  'https://einvestidor.estadao.com.br/ultimas/ibovespa-hoje-magazine-luiza-mglu3-dispara-9-braskem-brkm5-sofre/',
  'https://einvestidor.estadao.com.br/investimentos/dividendos-da-semana-bradesco-taesa-e-mais-6-empresas-pagam-acionistas-2/',
  'https://www.infomoney.com.br/mercados/7-acoes-sobem-mais-de-20-e-10-caem-mais-de-5-os-destaques-do-ibovespa-em-janeiro/'],
 'titulos': ['Agenda de dividendos de fevereiro: Petrobras, Itaú e Santander pagam neste mês',
  'Ibovespa hoje: Magazine Luiza (MGLU3) salta mais de 12%; só 4 ações fecham em queda',
  'Ibovespa hoje: Magazine Luiza (MGLU3) dispara 10% e lidera ganhos; Braskem (BRKM5) sofre',
  'Dividendos da semana: Bradesco (BBDC4), Banestes (BEES4), Itaú (ITUB4) e mais 9 empresas pagam...',
  '7 ações sobem mais de 20% e 10 caem m

In [4]:
ticker = 'BBDC3'
dados_google = TratarDadosNoticias(ticker, chrome_options)

In [6]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

ticker = 'BBDC3'
news_tratado = TratarDadosNoticias(ticker, chrome_options)
dados_new = news_tratado.clean_chat_html()

Sem mais páginas!
Poucas noticias encontrada para o ticker BBDC3 no Yahoo Finance. Buscando noticias no Google


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.24s/it]


In [8]:
dados_new.split('\n\n')

['\n New notice\n### Notícias sobre BBDC3',
 '#### Artigo 1: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '#### Artigo 2: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '#### Artigo 3: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '### Resumo',
 '* **BBDC3:** [Resumo das notícias sobre BBDC3]\n* **Notícias:**\n * [Notícia 1]\n * [Notícia 2]\n * [Notícia 3]',
 '### Detalhes',
 '* **Início da Operação:** [Data de início da operação]\n* **Inversão:** [Valor da inversão]\n* **Objetivo:** [Objetivo da empresa]\n* **Serviços:** [Serviços oferecidos pela empresa]\n* **Foco:** [Foco da empresa]',
 '### Notícias',
 '* [Notícia 1]: [Conteúdo da notícia]\n* [Notícia 2]: [Conteúdo da notícia]\n* [Notícia 3]: [Conteúdo da notícia]',
 '### Conclusão',
 '* **BBDC3:** [Resumo das notícias sobre BBDC3]',
 '#### Exemplo de conteúdo'

In [9]:
query = f'Qual é o sentimento sobre as notícias sobre a {ticker}'
dados_noticia = ChatSentimento(query=query, noticia=dados_new, modelo_llm="deepseek-r1-distill-llama-70b")

In [10]:
dados_noticia.split('\n\n')

['<think>\nOkay, I need to evaluate the overall sentiment of recent news about BBDC3 for an investment analysis. Let me start by going through each news article provided.',
 'First, I see several mentions of dividend payments and JCP (juros sobre capital próprio). These are positive indicators because they show that the company is generating enough profit to distribute to shareholders. High dividends can attract investors and indicate financial health.',
 "Next, there are articles about the company's strong financial results. For example, the lucro líquido (net income) was above expectations, with a 12% increase. This is a strong positive signal as it reflects good performance and stability.",
 'The expansion into international markets is another positive point. Entering new markets can lead to growth and diversification, which is great for long-term investment.',
 "The launch of a stock option plan is also positive. This can increase liquidity and attract more investors, which is bene

# Analise tecnica

In [15]:
trat_indi = TratandoDadosIndicadores(ticker=ticker, periodo= "18Y", intervalo="1mo" )


In [16]:
dados_indicadores =   trat_indi.indicadores_data_loader()


c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:828: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig0, xpt0, af0 = True, high[0], af
c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:829: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  _sar = [low[0] - (high - low).std()]
c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:834: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be t

In [17]:
dados_indicadores

[Document(metadata={'20 period SMA': 11.89498586654663, '100 period SMA': 12.328042049407959, '200 period SMA': 8.881622960567475, '14 period RSI': 44.05831839384031, 'MACD': -0.4836052105774389, 'SIGNAL': -0.5134500794645371, 'BB_UPPER': 14.279477388183793, 'BB_MIDDLE': 11.89498586654663, 'BB_LOWER': 9.510494344909468, 'pivot': 11.23797709607337, 's1': 10.658881950171024, 's2': 10.153276061596927, 's3': 9.57418091569458, 's4': 8.995085769792233, 'r1': 11.743582984647468, 'r2': 12.322678130549814, 'r3': 12.828284019123911, 'r4': 13.333889907698008, 'VWAP.': 10.502285948554505, '14 period ADX.': 13.103376736941206, 0: 9.926870539983447}, page_content='2023-10-01'),
 Document(metadata={'20 period SMA': 11.85486011505127, '100 period SMA': 12.375607585906982, '200 period SMA': 8.924924062490463, '14 period RSI': 52.005256820318266, 'MACD': -0.36686188026805233, 'SIGNAL': -0.48413243962524016, 'BB_UPPER': 14.156178073122218, 'BB_MIDDLE': 11.85486011505127, 'BB_LOWER': 9.553542156980322, 'p

In [8]:
response = ChatAnaliseTecnica(
    query=f"Qual é a tendencia sobre a empresa {ticker}",
    dados=dados_indicadores,
    modelo_llm="deepseek-r1-distill-llama-70b",
    stream=True
)

In [9]:
for chunk in response:
    print(chunk, end="", flush=True)

<think>
Okay, I need to analyze the trend for PETR4 using the given technical indicators. Let me go through each indicator step by step.

First, looking at the Moving Averages. The 20-period SMA is 30.14, the 100-period is 12.83, and the 200-period is 9.23. Since the 20-period is above the 100 and 200, it suggests an uptrend. Also, the 20-period SMA is increasing over time, which reinforces the uptrend.

Next, the RSI is at 64.58, which is below 70, so it's not overbought. That means there's still room for the price to go up without indicating a reversal.

The MACD is 4.62 and the Signal Line is 4.87. Since MACD is below the Signal Line, it might indicate a bearish crossover, but the values are still positive. This could mean a potential slowdown in the uptrend, but not a reversal yet.

Bollinger Bands show the upper band at 40.15, middle at 30.14, and lower at 20.12. The current price is near the middle band, which suggests moderate volatility. The bands are contracting, so volatility

In [ ]:
import yfinance as yf
ticker = "PETR4.SA"
acao = yf.Ticker(ticker,)
preco = acao.history(period="10Y", interval="1d")['Close']
preco

Date
2020-02-06 00:00:00-03:00     9.339462
2020-02-07 00:00:00-03:00     9.259444
2020-02-10 00:00:00-03:00     9.323457
2020-02-11 00:00:00-03:00     9.435479
2020-02-12 00:00:00-03:00     9.643520
                               ...    
2025-01-31 00:00:00-03:00    37.689999
2025-02-03 00:00:00-03:00    37.500000
2025-02-04 00:00:00-03:00    37.130001
2025-02-05 00:00:00-03:00    36.860001
2025-02-06 00:00:00-03:00    36.790001
Name: Close, Length: 1245, dtype: float64

In [49]:
dividendos = acao.dividends
dividendos

Date
2020-07-23 00:00:00-03:00    0.000461
2021-04-15 00:00:00-03:00    0.792834
2021-08-17 00:00:00-03:00    1.609911
2021-12-02 00:00:00-03:00    3.250487
2022-04-14 00:00:00-03:00    2.970249
2022-05-24 00:00:00-03:00    3.715490
2022-08-12 00:00:00-03:00    6.732003
2022-11-22 00:00:00-03:00    1.748708
2023-04-28 00:00:00-03:00    2.930163
2023-06-13 00:00:00-03:00    1.893576
2023-08-22 00:00:00-03:00    1.149304
2023-11-22 00:00:00-03:00    1.371248
2024-04-26 00:00:00-03:00    1.150024
2024-05-03 00:00:00-03:00    1.776022
2024-06-12 00:00:00-03:00    1.043242
2024-08-22 00:00:00-03:00    1.053200
2024-12-12 00:00:00-03:00    1.551743
2024-12-26 00:00:00-03:00    1.328207
Name: Dividends, dtype: float64

# Valuation modolo gordon

In [4]:
import numpy as np
class ValuationModoloGordon:
    def __init__(self, ticker: str, start_date_retorno: str="2004-01-01", end_date_retorno: str=datetime.today().strftime('%Y-%m-%d')):
        self.ticker = ticker
        self.start_date_retorno = start_date_retorno
        self.end_date_retorno = end_date_retorno
        self.dicionario_indicadores = {}
        self.dicionario_indicadores['ticker'] = ticker
    
    def acao(self):
        setando_ticker = yf.Ticker(self.ticker)
        setando_ticker.history(period="10Y", interval="1d")
        return setando_ticker

    def preco_historico(self):
        return self.acao().history(period="10Y", interval="1d")['Close']
        
    def g_sustainable(self):
        try:
            returnOnEquity = self.acao().info['returnOnEquity']
        except KeyError:
            print('Nao tem returnOnEquity')
            returnOnEquity = 0
        try:
            payoutRatio = self.acao().get_info()['payoutRatio']
        except KeyError:
            print('Nao tem payoutRatio')
            payoutRatio = 0
            
        g_sust= returnOnEquity *(1 - payoutRatio)
        
        self.dicionario_indicadores['g_sust'] = round(g_sust,4)
        
        return g_sust

    def beta(self):
        try:
            beta_acao = round(self.acao().info['beta'],4)
        except KeyError:
            print('Nao tem beta')
            beta_acao = 1
        self.dicionario_indicadores['beta'] = beta_acao
        return beta_acao
   
    def juros_livre(self):
        juros = (ip.timeseries("BMF12_SWAPDI36012")
               .rename(columns = {"VALUE ((% a.a.))": "swaps"})[['swaps']]
               .div(100)
               .iloc[-1]
               .swaps
        )
        self.dicionario_indicadores['juros_livre'] = round(juros,4)
        return juros
    
    def retorno_mercado(self):
        ibov = yf.download('^BVSP', start=self.start_date_retorno, end=self.end_date_retorno)
        porcentagem = ibov.pct_change()['Close']
        porcentagem.dropna(axis=0,inplace=True)
        compounded_growth = (1+porcentagem).prod()
        n_periods = porcentagem.shape[0]
        retorno_medio = compounded_growth**(252/n_periods)-1
        self.dicionario_indicadores['retorno_mercado'] = round(retorno_medio["^BVSP"],4)
        return retorno_medio["^BVSP"]
    
    def wacc(self):
        wacc = self.juros_livre() + self.beta() * (self.retorno_mercado()- self.juros_livre())
        self.dicionario_indicadores['wacc'] = round(wacc, 4)
        return wacc
    
    def dividendos(self):
        return self.acao().dividends
    
    def anual_dividendo(self):
        anual_dividendo = (
        pd.DataFrame(self.dividendos())
        .tz_localize(None)
        .assign(Year = lambda x: x.index.year)
        .groupby(['Year'])
        .agg({'Dividends' :'sum'})
        .iloc[:-1]
        )
        return anual_dividendo
        
    def d1(self):
        dividendo = self.anual_dividendo().median().Dividends
        self.dicionario_indicadores['dividendo_mediano'] = round(dividendo,3)
        return dividendo
    
    def preco_acao(self):
        
        pv= self.d1()/(self.wacc() - self.g_sustainable())
        
        preco_atual = self.preco_historico().values[-1]
        
        self.dicionario_indicadores['valuation_acao'] = round(pv,2)
        
        self.dicionario_indicadores['preco_atual'] = round(preco_atual,2)
        try:
            self.dicionario_indicadores['diferenca'] = round(((pv-preco_atual)/preco_atual)*100,2)
        except ValueError:
            self.dicionario_indicadores['diferenca'] = 'NaN'
        
        print(f"Valor presente da ação {self.ticker}: R$ {pv:.2f}")
        print(f"Cotação da ação {self.ticker}: R$ {preco_atual:.2f}")
        try:
            print(f"Diferença entre valuation e cotação: {((pv-preco_atual)/preco_atual)*100:.2f}%")
        except ValueError:
            print('Sem diferenca')
        if not np.isnan(pv):
            pv = round(pv)
        else:
            pv = 0
        return pv, self.dicionario_indicadores
    



In [ ]:
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
lista_valuation = []
for acao in acoes:
    acao_sa = acao + ".SA"
    valu = ValuationModoloGordon(acao_sa)
    try:
        _ , dicionario = valu.preco_acao()
        lista_valuation.append(dicionario)
    except KeyError:
        print("Erro ao obter dados da ação:", acao_sa)
    #print(dicionario)
    
valuation_acoes = pd.DataFrame(lista_valuation)
valuation_acoes.sort_values(by='diferenca', ascending=False).head(40)

,ticker,dividendo_mediano,juros_livre,beta,retorno_mercado,wacc,g_sust,valuation_acao,preco_atual,diferenca
147,TAEE4.SA,0.942,0.1502,0.283,0.0869,0.1323,0.1274,191.17,11.07,1626.94
146,TAEE3.SA,0.929,0.1502,0.283,0.0869,0.1323,0.1274,188.47,10.95,1621.21
136,QUAL3.SA,1.338,0.1502,0.852,0.0869,0.0963,-0.0081,12.82,1.78,620.15
20,AZZA3.SA,1.285,0.1502,0.811,0.0869,0.0989,0.0927,210.05,33.87,520.17
66,CMIG4.SA,0.370,0.1502,0.822,0.0869,0.0982,0.0927,67.12,10.98,511.26
86,ENGI4.SA,0.272,0.1502,0.451,0.0869,0.1217,0.1151,41.35,7.23,471.96
111,JHSF3.SA,0.290,0.1502,1.032,0.0869,0.0848,0.0701,19.65,3.88,406.50
51,BRSR6.SA,0.901,0.1502,0.680,0.0869,0.1072,0.0863,43.24,10.38,316.57
50,BRSR3.SA,0.901,0.1502,0.680,0.0869,0.1072,0.0863,43.24,12.00,260.33
85,ENGI3.SA,0.208,0.1502,0.451,0.0869,0.1217,0.1164,39.39,11.90,230.97


# Valuation fluxo de caixa descontado

In [156]:
from typing import Dict
class VariacaoReceita:
    def __init__(self,ticker: str, deflacionar_receita: bool = True):
        self.stock = yf.Ticker(ticker=ticker)
        self.deflacionar_receita = deflacionar_receita
        
    def financials(self):
        return self.stock.get_financials(freq='yearly')
        
    def pegando_inflacao(self) -> pd.DataFrame | None:
        ipca_raw = sidrapy.get_table(table_code='1737',
                                     territorial_level='1',
                                     ibge_territorial_code='all',
                                     variable='69',
                                     period='last%20472')
        if ipca_raw is not None:
            ipca_raw = ipca_raw[2:]
        else:
            ipca_raw = None
        return ipca_raw
    
    def modificando_datas_inflacao(self) -> pd.DataFrame | None:
        ipca_raw = self.pegando_inflacao()
        if ipca_raw is None:
            return None
        ipca_raw.loc[:, 'data'] = ipca_raw['D2C'].apply(lambda x: datetime.strptime(str(x), '%Y%m'))
        ipca_mes_doze = ipca_raw[ipca_raw['data'].dt.month == 12]
        ipca_mes_doze.loc[:, 'data_mes_ano'] = pd.to_datetime(ipca_mes_doze['data']).dt.strftime('%Y-%m')
        return ipca_mes_doze
    
    def receita_passada_dataframe(self) -> pd.DataFrame:
        receita_passada = pd.DataFrame(self.financials().loc['TotalRevenue'].iloc[::-1])
        receita_passada['mes_ano'] = pd.to_datetime(receita_passada.index).strftime('%Y-%m').values
        return receita_passada
    
    def pegando_inflacao_datas_receita(self) -> pd.DataFrame:
        receita_inflacao= self.receita_passada_dataframe()
        ipca_mes = self.modificando_datas_inflacao()
        if ipca_mes is not None and not pd.to_datetime(receita_inflacao.index).month.isin([6, 3, 9]).any():
            receita_inflacao['ipca'] = ipca_mes[ipca_mes['data_mes_ano'].isin(receita_inflacao['mes_ano'])]['V'].values.astype(float)
            receita_inflacao['TotalRevenue'] = receita_inflacao['TotalRevenue'].astype(float)

            receita_inflacao.reset_index(inplace=True)
        return receita_inflacao
    
    def inflacao_acumulada(self, data:pd.DataFrame) -> pd.DataFrame:
        inflacao_acumu = data.copy()
        inflacao_acumu['inflacao_acumulada'] = (1 + inflacao_acumu["ipca"].iloc[::-1].shift(1) / 100).cumprod().iloc[::-1].values
        inflacao_acumu.iloc[-1, -1] = 1
        return inflacao_acumu
    
    def deflacionando_receita(self, data: pd.DataFrame) -> pd.DataFrame:
        receita_deflacionada = data.copy()
        receita_deflacionada["TotalRevenueAjustado"] = receita_deflacionada['TotalRevenue'] * receita_deflacionada['inflacao_acumulada']
        return receita_deflacionada
    
    def pct_receita_normal(self, data:pd.DataFrame) -> pd.DataFrame:
        pct_data_normal = data.copy()
        pct_data_normal['receita_pct'] = pct_data_normal['TotalRevenue'].pct_change()
        return pct_data_normal
    
    def pct_receita_deflacionada(self, data:pd.DataFrame) ->pd.DataFrame:
        pct_dat_deflacionada = data.copy()
        pct_dat_deflacionada["receita_pct_deflacionado"] = pct_dat_deflacionada['TotalRevenueAjustado'].pct_change()
        return pct_dat_deflacionada
    
    def receita_crescimento_metricas(self) ->Dict[str, float]:
    
        receita_crescimento = self.pegando_inflacao_datas_receita()
        
        return_porcentagens = {}
    
        
        if self.deflacionar_receita and 'ipca' in receita_crescimento.columns:
            receita_crescimento_inflacao_acumulada = self.inflacao_acumulada(data=receita_crescimento)
            
            crescimento_deflacionando_receita = self.deflacionando_receita(receita_crescimento_inflacao_acumulada)
            
            pct_crescimento_receita_deflacionada = self.pct_receita_deflacionada(crescimento_deflacionando_receita)
            
            
            receita_crescimento = pct_crescimento_receita_deflacionada.copy()
            
            return_porcentagens['mean_deflacionada'] = pct_crescimento_receita_deflacionada['receita_pct_deflacionado'].mean()
            return_porcentagens['median_deflacionada'] = pct_crescimento_receita_deflacionada['receita_pct_deflacionado'].median() 
            
        
        
        receita_crescimento = self.pct_receita_normal(receita_crescimento)
        
        return_porcentagens['mean_normal'] = receita_crescimento['receita_pct'].mean()
        return_porcentagens['median_normal'] = receita_crescimento['receita_pct'].median()
        
        return return_porcentagens


                
        

In [279]:
class CustoCapitalProprio:
    def __init__(self, ticker: str, start_date_retorno: str="2004-01-01", end_date_retorno: str=datetime.today().strftime('%Y-%m-%d')):
        self.ticker = ticker
        self.start_date_retorno = start_date_retorno
        self.end_date_retorno = end_date_retorno
    
    def acao(self) -> yf.Ticker:
        setando_ticker = yf.Ticker(self.ticker)
        setando_ticker.history(period="10Y", interval="1d")
        return setando_ticker

    def preco_historico(self) -> pd.Series:
        return self.acao().history(period="10Y", interval="1d")['Close']
        
    def beta(self) -> float:
        try:
            beta_acao = round(self.acao().info['beta'],4)
        except KeyError:
            print('Nao tem beta')
            beta_acao = 1
        return beta_acao
   
    def juros_livre(self) -> float:
        juros = (ip.timeseries("BMF12_SWAPDI36012")
               .rename(columns = {"VALUE ((% a.a.))": "swaps"})[['swaps']]
               .div(100)
               .iloc[-1]
               .swaps
        )
        return juros
    
    def retorno_mercado(self):
        ibov = yf.download('^BVSP', start=self.start_date_retorno, end=self.end_date_retorno)
        porcentagem = ibov.pct_change()['Close']
        porcentagem.dropna(axis=0,inplace=True)
        compounded_growth = (1+porcentagem).prod()
        n_periods = porcentagem.shape[0]
        retorno_medio = compounded_growth**(252/n_periods)-1
        return retorno_medio["^BVSP"]
    
    def wacc(self) -> float:
        wacc = self.juros_livre() + self.beta() * (self.retorno_mercado()- self.juros_livre())
        return round(wacc,2)

In [287]:
ibov = yf.download('^BVSP', start=wac.start_date_retorno, end=wac.end_date_retorno)
porcentagem = ibov.pct_change()['Close']
porcentagem.dropna(axis=0,inplace=True)
compounded_growth = (1+porcentagem).prod()
n_periods = porcentagem.shape[0]
retorno_medio = compounded_growth**(252/n_periods)-1
retorno_medio["^BVSP"]

[*********************100%***********************]  1 of 1 completed

0.08618870117824207

In [283]:
wac = CustoCapitalProprio(ticker='PETR4.SA')
wac.wacc()

[*********************100%***********************]  1 of 1 completed


0.09

In [284]:
wac.retorno_mercado()

[*********************100%***********************]  1 of 1 completed


0.08618870117824207

In [ ]:
variacao_receita = VariacaoReceita(ticker='AGRO3.SA')   
var = variacao_receita.receita_crescimento_metricas()
var

In [ ]:
import numpy as np
class IndicadoresFinanceiros:
    def __init__(self, ticker: str, margem_ebit_mediana: bool = True, deflacionar_receita: bool = True, percentual_imposto_mediana: bool = True, depreciacao_capex_mediana: bool = True, capex_receita_mediana: bool = True):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker=self.ticker)
        self.financials = self.stock.financials
        self.balance_sheet = self.stock.balance_sheet
        self.cashflow = self.stock.cashflow
        self.margem_ebit_mediana =  margem_ebit_mediana
        self.deflacionar_receita = deflacionar_receita
        self.percentual_imposto_mediana = percentual_imposto_mediana
        self.depreciacao_capex_mediana = depreciacao_capex_mediana
        self.capex_receita_mediana = capex_receita_mediana
        
    def ultima_receita(self) -> pd.Series:
        receita_ano = self.financials.loc['Total Revenue'].iloc[0]
        return receita_ano
    
    def variacao_receita_ultimos_anos(self) -> Dict[str, float]:
        variacao_receita = VariacaoReceita(ticker=self.ticker, deflacionar_receita=self.deflacionar_receita)   
        porcentagem_receita = variacao_receita.receita_crescimento_metricas()
        return porcentagem_receita
        
    def margem_ebit(self) -> np.float64:
        ebit_ultimos_anos = self.financials.loc['EBIT']
        receita_ultimos_anos = self.financials.loc['Total Revenue']
        margem_ebit = ebit_ultimos_anos / receita_ultimos_anos
        if self.margem_ebit_mediana:
            margem_ebit = round(margem_ebit.median(),2)
        else:
            margem_ebit = round(margem_ebit.mean(),2) 
        return np.float64(round(margem_ebit.item() if isinstance(margem_ebit, pd.Series) else margem_ebit, 2))
    
    def percentual_imposto(self):
        ebit_ultimos_anos = self.financials.loc['EBIT']
        
        imposto_ultimos_anos = self.financials.loc['Tax Provision']
        
        if isinstance(ebit_ultimos_anos, pd.DataFrame):
            ebit_ultimos_anos = ebit_ultimos_anos.iloc[:, 0]
        if isinstance(imposto_ultimos_anos, pd.DataFrame):
            imposto_ultimos_anos = imposto_ultimos_anos.iloc[:, 0]
        
        tax_ebit = imposto_ultimos_anos/ebit_ultimos_anos
        
        tax_ebit_sem_zero = tax_ebit[tax_ebit > 0]
        
        if tax_ebit_sem_zero.empty:
            return round(tax_ebit.mean(), 4)
        return round(
        tax_ebit_sem_zero.median() if self.percentual_imposto_mediana else tax_ebit_sem_zero.mean(),
        4
        )
    
    def depreciacao_capex(self):
        depreciacao_amortizacao = self.cashflow.loc['Depreciation And Amortization']
        capex = self.cashflow.loc['Capital Expenditure']
        
        depre_capex = depreciacao_amortizacao/capex
        
        if self.depreciacao_capex_mediana:
            return abs(depre_capex.median())
        else:
            return abs(depre_capex.mean())
        
    def capex_receita(self):
        capex = self.cashflow.loc['Capital Expenditure']
        receita = self.financials.loc['Total Revenue']
        
        capex_recei = capex / receita
        
        if self.capex_receita_mediana:
            return abs(capex_recei.median())
        else:
            return abs(capex_recei.mean())
        
    
    
        
        
        
        
        
        
        
        
    

In [269]:
ind = IndicadoresFinanceiros(ticker='AGRO3.SA', percentual_imposto_mediana=True)
print('Ultima Receita',ind.ultima_receita())
print('Variacao Receita',ind.variacao_receita_ultimos_anos())
print('Margem EBIT',ind.margem_ebit())
print('Percentual Imposto',ind.percentual_imposto())
print('Depreciacao Capex',ind.depreciacao_capex())
print('Capex Receita',ind.capex_receita())

Ultima Receita 1018410000.0
Variacao Receita {'mean_normal': 0.23323418320771924, 'median_normal': -0.12208797318038733}
Margem EBIT 0.37
Percentual Imposto 0.1029
Depreciacao Capex 1.5403677953962518
Capex Receita 0.04386125373507237


C:\Users\jefer\AppData\Local\Temp\ipykernel_29756\2641360973.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipca_mes_doze.loc[:, 'data_mes_ano'] = pd.to_datetime(ipca_mes_doze['data']).dt.strftime('%Y-%m')
C:\Users\jefer\AppData\Local\Temp\ipykernel_29756\2641360973.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pct_data_normal['receita_pct'] = pct_data_normal['TotalRevenue'].pct_change()


In [270]:
wacc = info.get('wacc', None)

In [271]:
wacc

In [254]:
stock = yf.Ticker('AGRO3.SA')
info = stock.info
financials = stock.financials
balance_sheet = stock.balance_sheet
cashflow = stock.cashflow

In [ ]:
import yfinance as yf

def get_financial_indicators(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
    
    capex = cashflow.loc['Capital Expenditures'].iloc[0]
    depreciacao = cashflow.loc['Depreciation'].iloc[0]
    depreciacao_capex = depreciacao / capex
    capex_da_receita = capex / receita_ano
    
    wacc = info.get('wacc', None)  # Nem sempre disponível no Yahoo Finance
    numero_de_acoes = info['sharesOutstanding']
    divida = balance_sheet.loc['Total Debt'].iloc[0]
    disponivel = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
    ativos_nao_operacionais = balance_sheet.loc['Other Assets'].iloc[0]
    passivos_circulantes = balance_sheet.loc['Other Current Liabilities'].iloc[0]
    taxa_crecimento_perpetuidade = 0.02  # Padrão para perpetuidade, pode ser ajustado
    
    recebiveis = balance_sheet.loc['Net Receivables'].iloc[0]
    estoque = balance_sheet.loc['Inventory'].iloc[0]
    outros_ac = balance_sheet.loc['Other Current Assets'].iloc[0]
    outros_pc = balance_sheet.loc['Other Current Liabilities'].iloc[0]
    fornecedores = balance_sheet.loc['Accounts Payable'].iloc[0]
    
    necessidade_capital_de_giro = (recebiveis + estoque + outros_ac) - (outros_pc + fornecedores)
    
    return {
        'anos_projecao': anos_projecao,
        'receita_ano': receita_ano,
        'porcenta_crescimento_receita': porcenta_crescimento_receita,
        'margem_ebit': margem_ebit,
        'imposto_porcentagem': imposto_porcentagem,
        'depreciacao_capex': depreciacao_capex,
        'capex_da_receita': capex_da_receita,
        'wacc': wacc,
        'numero_de_acoes': numero_de_acoes,
        'divida': divida,
        'disponivel': disponivel,
        'ativos_nao_operacionais': ativos_nao_operacionais,
        'passivos_circulantes': passivos_circulantes,
        'taxa_crecimento_perpetuidade': taxa_crecimento_perpetuidade,
        'necessidade_capital_de_giro': necessidade_capital_de_giro
    }

# Exemplo de uso
empresa = "PETR4.SA"  # Petrobras, por exemplo
dados = get_financial_indicators(empresa)
print(dados)


In [61]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
def quantidade_acoes(ticker):
    site = f"https://www.investsite.com.br/quantidade_acoes.php?cod_negociacao={ticker}"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    total_acoes = soup.find('table', id='quantidade_acoes').find('tbody').find_all('tr')[-1].find_all('td')[1].text
    return total_acoes

In [72]:
print(ticker)
quantidade_acoes('PETR4')



PETR4.SA


'13.044.496.930'

In [1]:
import yfinance as yf
ticker = "PETR4.SA"
acao = yf.Ticker(ticker)
preco = acao.history(period="10Y", interval="1d")['Close']

def quantidade_acoes():
    df = acao.quarterly_balance_sheet.loc[acao.quarterly_balance_sheet.index == 'Share Issued']
    return df.iloc[:,0].values[0]

quantidade_acoes()

13044496930.0

In [2]:
def divida_liquida():
    df = acao.quarterly_balance_sheet.loc[acao.quarterly_balance_sheet.index == 'Net Debt']
    return df.iloc[:,0].values[0]

divida_liquida()

17062000000.0

In [10]:
tic = 'PETR4'
data_inicio = '2020-01-01'
dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)

In [11]:
dados_petr = dados_fundamentalistas.dados_fundamentalistas_completo()

In [12]:
def margem_ebit(dados):
    return dados[['margem_ebit','datas']].iloc[::-1][-5:].mean()['margem_ebit']
    

margem_ebit(dados_petr)

0.353

In [13]:
receita_liquida = dados_petr[['receita_liquida','datas']].iloc[::-1]
receita_liquida['datas_ano'] = pd.to_datetime(receita_liquida['datas']).dt.year

In [14]:
receita_liquida

,receita_liquida,datas,datas_ano
18,75470.0,2020-03-31,2020
17,50900.0,2020-06-30,2020
16,70730.0,2020-09-30,2020
15,74970.0,2020-12-31,2020
14,86170.0,2021-03-31,2021
13,110710.0,2021-06-30,2021
12,121590.0,2021-09-30,2021
11,134190.0,2021-12-31,2021
10,141640.0,2022-03-31,2022
9,170960.0,2022-06-30,2022


In [36]:
def receita_ultimo_ano(dados):
    receita_liquida = dados[['receita_liquida','datas']].iloc[::-1]
    receita_liquida['datas_ano'] = pd.to_datetime(receita_liquida['datas']).dt.year
    receita_sum =receita_liquida.groupby('datas_ano').agg({'receita_liquida': 'sum', 'datas_ano': 'count'}).rename(columns = {'datas_ano': 'numero_quartes', 'receita_liquida': 'receita_liquida_total'})
    if receita_sum['numero_quartes'].iloc[-1] == 4:
        receita_sum = receita_sum['receita_liquida_total'].iloc[-1]
    else:
        receita_sum = receita_sum['receita_liquida_total'].iloc[-2]
    return receita_sum

In [118]:
def receita_ultimo_ano(dados: pd.DataFrame) -> float:
    """
    Retorna a receita líquida total do último ano completo.

    Parâmetros:
        dados (pd.DataFrame): DataFrame com as informações de receita.

    Retorna:
        float: Receita líquida total do último ano completo.
    """
    # Inverter a ordem das linhas para começar pelo último ano
    dados = dados.iloc[::-1]

    # Extrair o ano das datas
    anos = pd.to_datetime(dados['datas']).dt.year

    # Agrupar por ano e calcular a soma da receita líquida
    receita_por_ano = dados.groupby(anos)['receita_liquida'].sum()

    # Verificar se o último ano tem 4 trimestres
    if receita_por_ano.index[-1] in anos.value_counts()[anos.value_counts() == 4].index:
        return receita_por_ano.iloc[-1]
    else:
        return receita_por_ano.iloc[-2]

receita_ultimo_ano(dados_petr)*1000000

512000000000.0

In [158]:
def crescimento_receita(dados: pd.DataFrame) -> float:
    crescimento_receita = dados[['receita_liquida','datas']].iloc[::-1].groupby(pd.to_datetime(dados['datas']).dt.year).agg({'receita_liquida': 'sum', 'datas': 'last'})
    crescimento_receita['receita_liquida_pc'] = crescimento_receita['receita_liquida'].pct_change().dropna()
    return round(crescimento_receita['receita_liquida_pc'].mean(),3)

crescimento_receita(dados_petr)

0.15

In [148]:
def capex_receita(dados: pd.DataFrame) -> float:
    receita_ano = dados[['datas', 'receita_liquida']].iloc[::-1].groupby(pd.to_datetime(dados['datas']).dt.year).agg({'receita_liquida': 'sum', 'datas': 'last'})
    receita_ano['capex'] = dados[dados[['datas', 'capex_doze_meses']].iloc[::-1]['datas'].isin(receita_ano['datas'])]['capex_doze_meses'].values
    return round(abs((receita_ano['capex']/receita_ano['receita_liquida']).mean()),3)

capex_receita(dados_petr)   

C:\Users\jefer\AppData\Local\Temp\ipykernel_29656\638925661.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  receita_ano['capex'] = dados[dados[['datas', 'capex_doze_meses']].iloc[::-1]['datas'].isin(receita_ano['datas'])]['capex_doze_meses'].values


0.124

In [117]:
def capex_doze_meses(dados: pd.DataFrame) -> float:
    return dados[['capex_doze_meses','datas']].iloc[::-1].iloc[-1]['capex_doze_meses']

capex_doze_meses(dados_petr)*1000000

-71740000000.0

In [116]:
def divida_bruta(dados: pd.DataFrame) -> float:
    return dados[['divida_bruta','datas']].iloc[::-1].iloc[-1]['divida_bruta']

divida_bruta(dados_petr)*1000000

322160000000.0

In [114]:
def caixa_equivalentes_caixa(dados: pd.DataFrame) -> float:
    return dados[['caixa_equivalentes_caixa','datas']].iloc[::-1].iloc[-1]['caixa_equivalentes_caixa']

caixa_equivalentes_caixa(dados_petr)*1000000

47370000000.0

In [177]:
def capital_giro(ticker: str, options: webdriver.ChromeOptions) -> float:
    navegador = webdriver.Chrome(options=options)
    site = f"https://www.investsite.com.br/formula.php?cod_negociacao={ticker}&ratio=Pre%C3%A7o%20Ordin%C3%A1ria%2FCapital%20de%20Giro%20Atual&data_ratio=20250205&tipodata=atual&cons_ratio=2&periodo=NA"

    navegador.get(site)

    # Pegando o elemento
    elemento = navegador.find_element(By.XPATH, '//*[@id="insere_aqui"]')

    padrao = r"Capital de Giro\s*=\s*([-−]?\s*\d{1,3}(?:\.\d{3})*,\d{2})"
    
    resultado = re.findall(padrao, elemento.text)
    
    if resultado:
        capital_giro = float(resultado[0].replace(".", "").replace(",", "."))
        return capital_giro
    else:
        return 0.0

In [210]:
capital_giro("FIQE3", chrome_options)

276668000.0

**Resumo**

Método	O que retorna?

.get_income_stmt() / .income_stmt	Demonstração de Resultados

.get_balance_sheet() / .balance_sheet	Balanço Patrimonial

.get_cashflow() / .cashflow	Fluxo de Caixa

.get_earnings() / .earnings	Ganhos (Earnings)

.get_earnings_dates() / .earnings_dates	Datas de Divulgação de Resultados

.calendar	Eventos futuros da empresa

.get_sec_filings() / .sec_filings	Documentos enviados à SEC

In [263]:
def receita_ultimo_ano() -> float:
    return acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[0]

receita_ultimo_ano()

102409000000.0

In [274]:
acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[::-1].pct_change()

C:\Users\jefer\AppData\Local\Temp\ipykernel_29656\2999328729.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[::-1].pct_change()


2020-12-31         NaN
2021-12-31    0.564108
2022-12-31    0.482433
2023-12-31   -0.177266
Name: TotalRevenue, dtype: float64

In [278]:
acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[::-1].pct_change()

C:\Users\jefer\AppData\Local\Temp\ipykernel_29656\2999328729.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[::-1].pct_change()


2020-12-31         NaN
2021-12-31    0.564108
2022-12-31    0.482433
2023-12-31   -0.177266
Name: TotalRevenue, dtype: float64

In [250]:
acao.get_income_stmt()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
TaxEffectOfUnusualItems,-273420000.0,630382000.0,-397281806.908769,-4959920000.0
TaxRateForCalcs,0.294,0.313,0.291904,0.34
NormalizedEBITDA,53226000000.0,68040000000.0,46258000000.0,31578000000.0
TotalUnusualItems,-930000000.0,2014000000.0,-1361000000.0,-14588000000.0
TotalUnusualItemsExcludingGoodwill,-930000000.0,2014000000.0,-1361000000.0,-14588000000.0
...,...,...,...,...
SalariesAndWages,3547000000.0,2655000000.0,2984000000.0,101000000.0
GrossProfit,53974000000.0,64988000000.0,40802000000.0,24488000000.0
CostOfRevenue,48435000000.0,59486000000.0,43164000000.0,29195000000.0
TotalRevenue,102409000000.0,124474000000.0,83966000000.0,53683000000.0


In [262]:
acao.get_income_stmt(freq='yearly').loc['TotalRevenue'].iloc[0]

102409000000.0

In [245]:
acao.get_balance_sheet(pretty=True,freq='yearly')

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,104359669.0,295669.0,295669.0,295669.0
Preferred Shares Number,NaN,NaN,5601969879.0,NaN
Ordinary Shares Number,12940137261.0,13044201261.0,13044201261.0,13044201261.0
Share Issued,13044496930.0,13044496930.0,13044496930.0,13044496930.0
Net Debt,16074000000.0,21958000000.0,25233000000.0,42177000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,15546000000.0,10769000000.0,11117000000.0,12370000000.0
Other Short Term Investments,2819000000.0,2773000000.0,650000000.0,659000000.0
Cash And Cash Equivalents,12727000000.0,7996000000.0,10467000000.0,11711000000.0
Cash Equivalents,12624000000.0,7780000000.0,10168000000.0,11159000000.0


In [242]:
acao.get_balance_sheet(freq='yearly')[acao.get_balance_sheet(freq='yearly').index == 'AccumulatedDepreciation']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
AccumulatedDepreciation,-120488000000.0,-100792000000.0,-91077000000.0,-100673000000.0


In [225]:
acao.get_financials().index

Index(['TaxEffectOfUnusualItems', 'TaxRateForCalcs', 'NormalizedEBITDA',
       'TotalUnusualItems', 'TotalUnusualItemsExcludingGoodwill',
       'NetIncomeFromContinuingOperationNetMinorityInterest',
       'ReconciledDepreciation', 'ReconciledCostOfRevenue', 'EBITDA', 'EBIT',
       'NetInterestIncome', 'InterestExpense', 'InterestIncome',
       'NormalizedIncome', 'NetIncomeFromContinuingAndDiscontinuedOperation',
       'TotalExpenses', 'RentExpenseSupplemental', 'DilutedAverageShares',
       'BasicAverageShares', 'DilutedEPS', 'BasicEPS',
       'DilutedNIAvailtoComStockholders', 'NetIncomeCommonStockholders',
       'NetIncome', 'MinorityInterests',
       'NetIncomeIncludingNoncontrollingInterests',
       'NetIncomeDiscontinuousOperations', 'NetIncomeContinuousOperations',
       'TaxProvision', 'PretaxIncome', 'OtherIncomeExpense',
       'OtherNonOperatingIncomeExpenses', 'SpecialIncomeCharges',
       'GainOnSaleOfPPE', 'GainOnSaleOfBusiness', 'OtherSpecialCharges',
      

In [236]:
acao.get_financials(freq='yearly').loc[acao.get_financials().index == 'DepreciationAmortizationDepletionIncomeStatement']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
DepreciationAmortizationDepletionIncomeStatement,732000000.0,894000000.0,696000000.0,653000000.0


In [237]:
acao.get_financials(freq='yearly').loc[acao.get_financials().index == 'TaxProvision']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
TaxProvision,10401000000.0,16770000000.0,8239000000.0,-1174000000.0


In [230]:
acao.get_financials(freq='yearly').loc[acao.get_financials().index == 'OtherTaxes']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
OtherTaxes,686000000.0,353000000.0,-112000000.0,-2435000000.0


In [235]:
acao.get_financials(freq='yearly').loc[acao.get_financials().index == 'ProvisionForDoubtfulAccounts']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
ProvisionForDoubtfulAccounts,22000000.0,58000000.0,-12000000.0,-2000000.0


In [228]:
acao.get_balance_sheet().loc[acao.get_balance_sheet().index == 'AccumulatedDepreciation']

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
AccumulatedDepreciation,-120488000000.0,-100792000000.0,-91077000000.0,-100673000000.0


In [234]:
acao.get_balance_sheet(freq='yearly')

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
TreasurySharesNumber,104359669.0,295669.0,295669.0,295669.0
PreferredSharesNumber,NaN,NaN,5601969879.0,NaN
OrdinarySharesNumber,12940137261.0,13044201261.0,13044201261.0,13044201261.0
ShareIssued,13044496930.0,13044496930.0,13044496930.0,13044496930.0
NetDebt,16074000000.0,21958000000.0,25233000000.0,42177000000.0
...,...,...,...,...
CashCashEquivalentsAndShortTermInvestments,15546000000.0,10769000000.0,11117000000.0,12370000000.0
OtherShortTermInvestments,2819000000.0,2773000000.0,650000000.0,659000000.0
CashAndCashEquivalents,12727000000.0,7996000000.0,10467000000.0,11711000000.0
CashEquivalents,12624000000.0,7780000000.0,10168000000.0,11159000000.0
